# Prompt Optimization

In this notebook, we will use `mlflow.genai.optimize_prompts` to optimize our lease extraction prompt. After optimization, we will evaluate the performance using the held-out evaluation set.

In [ ]:
import mlflow
from mlflow.genai.optimize import GepaPromptOptimizer
import pandas as pd
from utils import (
    setup_mlflow,
    load_config,
    create_predict_fn,
    create_fuzzy_scorers,
    EXTRACTION_FIELDS,
)

setup_mlflow()
config = load_config()

print(f"User: {config['user']}")
print(f"Experiment: {config['mlflow_experiment_name']}")

## Load data

In [ ]:
# Load full training data from Delta table
train_table_name = f"{config['catalog']}.{config['schema']}.prompt_opt_training"
train_sdf = spark.read.table(train_table_name)
print(f"Total training records: {train_sdf.count()}")

# Load eval dataset from MLflow
eval_dataset_name = f"{config['catalog']}.{config['schema']}.prompt_opt_eval"
eval_dataset = mlflow.genai.datasets.get_dataset(eval_dataset_name)
eval_records = eval_dataset.to_df()[["inputs", "expectations"]].to_dict(
    orient="records"
)
print(f"Total eval records: {len(eval_records)}")

## Optimizing prompt with GEPA

In [ ]:
# Get training data
train_pdf = train_sdf.toPandas()

# Convert to records format
train_records = []
for _, row in train_pdf.iterrows():
    record = {
        "inputs": {"query": row.get("request", "")},
        "expectations": {"expected_response": row.get("labels", "")},
    }
    train_records.append(record)

print(f"Records for optimization: {len(train_records)}")

In [ ]:
base_prompt_uri = (
    f"prompts:/{config['catalog']}.{config['schema']}.lease_extraction_prompt/1"
)

predict_fn_base = create_predict_fn(base_prompt_uri)

# Create fuzzy scorers for optimization
fuzzy_scorers = create_fuzzy_scorers(threshold=0.7)


# Aggregation function to average all fuzzy match scores
def aggregation_fn(scores: dict) -> float:
    """Average all fuzzy match scores across fields."""
    total = 0.0
    count = 0
    for field in EXTRACTION_FIELDS:
        feedback = scores.get(field)
        if feedback is not None:
            total += float(feedback.value)
            count += 1
    return total / count if count > 0 else 0.0


with mlflow.start_run(run_name="Optimization"):
    optimization = mlflow.genai.optimize_prompts(
        predict_fn=predict_fn_base,
        train_data=train_records,
        prompt_uris=[base_prompt_uri],
        optimizer=GepaPromptOptimizer(
            reflection_model=f"databricks:/{config['optimizer_endpoint']}",
            max_metric_calls=len(train_records) * 3,
        ),
        scorers=fuzzy_scorers,
        aggregation=aggregation_fn,
    )

## Evaluate optimized prompt

In [ ]:
optimized_prompt_uri = (
    f"prompts:/{config['catalog']}.{config['schema']}.lease_extraction_prompt/4"
)

predict_fn_new = create_predict_fn(optimized_prompt_uri)

In [ ]:
# Create fresh scorers for eval
eval_scorers = create_fuzzy_scorers(threshold=0.7)

with mlflow.start_run(run_name="Eval Round"):
    eval_results = mlflow.genai.evaluate(
        data=eval_records,
        predict_fn=predict_fn_new,
        scorers=eval_scorers,
    )

# Print per-field scores and overall average
mean_scores = []
for key, value in eval_results.metrics.items():
    if "/mean" in key:
        print(f"{key}: {value:.2%}")
        mean_scores.append(value)

if mean_scores:
    print(f"\nOverall average: {sum(mean_scores) / len(mean_scores):.2%}")

With 70 training samples, the GEPA-optimized prompt was able to achieve an increase in average accuracy by more than **16%** across the extraction fields.